In [1]:
pip install python-telegram-bot pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 11.8 MB/s eta 0:00:00


In [2]:
pip install python-telegram-bot --upgrade

In [ ]:
'25/4oho'.split('.')

['25/4oho']

In [75]:
from telegram import Update
from telegram.ext import (
    Application,
    CommandHandler,
    ConversationHandler,
    MessageHandler,
    filters,
    ContextTypes,
)

import requests
import xml.etree.ElementTree as ET
import re
from datetime import datetime

def get_exchange_rate(date: str, currency_code: str) -> str:
    """
    Получает курс валюты из ЦБ РФ.

    :param date: Дата в формате "ДД.ММ.ГГГГ"
    :param currency_code: Код валюты (например, 'USD')
    :return: Курс валюты или сообщение об ошибке.
    """
    example1=date.split(".")
    if len(example1) != 3:
        return "Некорректная дата"
    if not ((1<=int(example1[0])<=31) and (len(example1[0])==2)):
      return "Некорректная дата"
    if not ((1<=int(example1[1])<=12) and (len(example1[1])==2)):
      return "Некорректная дата"
    if not ((1992<=int(example1[2])<=2024) and (len(example1[2])==4)):
      return "Некорректная дата"
    url = f"http://www.cbr.ru/scripts/XML_daily.asp?date_req={date}"
    response = requests.get(url)

    if response.status_code != 200:
        return f"Ошибка запроса: {response.status_code}"

    # Парсим XML
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()

    for valute in root.findall('Valute'):
        char_code = valute.find('CharCode').text
        if char_code == currency_code:
            nominal = int(valute.find('Nominal').text)
            value = float(valute.find('Value').text.replace(',', '.'))
            return f"Курс {currency_code} на {date}: {value / nominal} RUB"

    return f"Валюта {currency_code} не найдена на дату {date}, воспользуйтесь функцией /available."

import requests
import xml.etree.ElementTree as ET

import requests
import xml.etree.ElementTree as ET

import requests
import xml.etree.ElementTree as ET

import requests
import xml.etree.ElementTree as ET

def convert_currency(amount: int, from_currency: str, to_currency: str, date: str) -> str:
    """
    Конвертирует валюту на основе курса ЦБ РФ.
    :param amount: Сумма для конвертации.
    :param from_currency: Код исходной валюты (например, 'USD').
    :param to_currency: Код целевой валюты (например, 'RUB').
    :param date: Дата в формате "ДД.ММ.ГГГГ".
    :return: Результат конвертации или сообщение об ошибке.
    """
    url = f"http://www.cbr.ru/scripts/XML_daily.asp?date_req={date}"
    response = requests.get(url)

    if response.status_code != 200:
        return f"Ошибка запроса: {response.status_code}"

    # Парсим XML
    tree = ET.ElementTree(ET.fromstring(response.content))
    root = tree.getroot()

    # Обработка конверсии в рубль
    if to_currency == "RUB":
        for valute in root.findall('Valute'):
            char_code = valute.find('CharCode')
            if char_code is not None and char_code.text == from_currency:
                nominal = int(valute.find('Nominal').text)
                value = float(valute.find('Value').text.replace(',', '.'))
                result = amount * (value / nominal)  # Конвертируем из валюты в рубли
                return f"{amount} {from_currency} = {result:.2f} RUB по курсу на {date}"
        return f"Валюта {from_currency} не найдена на дату {date}, воспользуйтесь функцией /available."

    # Обработка конверсии из рубля
    if from_currency == "RUB":
        for valute in root.findall('Valute'):
            char_code = valute.find('CharCode')
            if char_code is not None and char_code.text == to_currency:
                nominal = int(valute.find('Nominal').text)
                value = float(valute.find('Value').text.replace(',', '.'))
                result = amount * (nominal / value)  # Конвертируем из рубля в целевую валюту
                return f"{amount} RUB = {result:.2f} {to_currency} по курсу на {date}"
        return f"Валюта {to_currency} не найдена на дату {date}, воспользуйтесь функцией /available."

    # Обработка других валют
    from_currency_data = None
    to_currency_data = None

    for valute in root.findall('Valute'):
        char_code = valute.find('CharCode')
        if char_code is not None:
            if char_code.text == from_currency:
                nominal_from = int(valute.find('Nominal').text)
                value_from = float(valute.find('Value').text.replace(',', '.'))
                from_currency_data = (nominal_from, value_from)
            elif char_code.text == to_currency:
                nominal_to = int(valute.find('Nominal').text)
                value_to = float(valute.find('Value').text.replace(',', '.'))
                to_currency_data = (nominal_to, value_to)

    if from_currency_data and to_currency_data:
        nominal_from, value_from = from_currency_data
        nominal_to, value_to = to_currency_data
        scale_from = value_from / nominal_from
        scale_to = value_to / nominal_to
        result = (amount * scale_from) / scale_to
        return f"{amount} {from_currency} = {result:.2f} {to_currency} по курсу на {date}"

    return f"Валюта {from_currency} или {to_currency} не найдена на дату {date}, воспользуйтесь функцией /available."
##############################################################################################################################################
def is_valid_date(date: str) -> bool:
    """Проверяет, соответствует ли дата формату ДД.ММ.ГГГГ (только цифры и точки)."""
    # Проверяем, что дата состоит только из цифр и точек, и в правильном формате ДД.ММ.ГГГГ
    if re.match(r"^\d{2}\.\d{2}\.\d{4}$", date):
        return True
    return False

def available(date: str) -> str:
    if not is_valid_date(date):
        return "Введите дату в формате ДД.ММ.ГГГГ"  # Выводим ошибку, если формат неверный

    url = f"http://www.cbr.ru/scripts/XML_daily.asp?date_req={date}"
    response = requests.get(url)

    if response.status_code != 200:
        return f"Ошибка запроса: {response.status_code}"

    try:
        tree = ET.ElementTree(ET.fromstring(response.content))
        root = tree.getroot()
    except ET.ParseError:
        return "Ошибка обработки ответа от сервера."

    # Проверка на пустой или некорректный ответ
    if root is None:
        return "Не удалось получить данные о валюте. Проверьте дату или повторите запрос позже."

    # Получение списка доступных валют
    currencies = []
    for valute in root.findall('Valute'):
        char_code = valute.find('CharCode')
        if char_code is not None:
            currencies.append(char_code.text)

    if not currencies:
        return f"Не удалось найти валюты на дату {date}. Возможно, на эту дату нет данных."

    return f"Доступные валюты на {date}: " + ", ".join(currencies)
############################################################################################################################################
# def convert_currency(amount: int, from_currency: str, to_currency: str, date: str) -> str:
#     """
#     Получает курс валюты из ЦБ РФ.

#     :param date: Дата в формате "ДД.ММ.ГГГГ"
#     :param from_currency: Код валюты (например, 'USD')
#     :param to_currency: Код валюты (например, 'USD')
#     :return: Курс валюты или сообщение об ошибке.
#     """
#     url = f"http://www.cbr.ru/scripts/XML_daily.asp?date_req={date}"
#     response = requests.get(url)

#     if response.status_code != 200:
#         return f"Ошибка запроса: {response.status_code}"

#     # Парсим XML
#     tree = ET.ElementTree(ET.fromstring(response.content))
#     root = tree.getroot()
#     err = True
#     scale = 1
#     for valute in root.findall('Valute'):
#         char_code = valute.find('CharCode').text
#         if char_code == to_currency:
#             err=False
#             nominal = int(valute.find('Nominal').text)
#             value = float(valute.find('Value').text.replace(',', '.'))
#             scale = value / nominal
#     if err:
#      return  f"Валюта {to_currency} не найдена на дату {date}."

#     for valute in root.findall('Valute'):
#         char_code = valute.find('CharCode').text
#         if char_code == from_currency:
#             nominal = int(valute.find('Nominal').text)
#             value = float(valute.find('Value').text.replace(',', '.'))
#             res = (value / nominal * amount) / scale
#             return f'{amount} валюты {from_currency} = {res} {to_currency}'

#     return f"Валюта {from_currency} не найдена на дату {date}."

# Состояния разговора для команд
WAITING_FOR_RATE_DATE = 1
WAITING_FOR_RATE_CURRENCY = 2
WAITING_FOR_CONVERT_DATE = 3
WAITING_FOR_CONVERT_AMOUNT = 4
WAITING_FOR_CONVERT_FROM = 5
WAITING_FOR_CONVERT_TO = 6

def exchanger(date: str, currency_code: str) -> str:
    return get_exchange_rate(date, currency_code)


# Обработчик команды /start
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    await update.message.reply_text(
        "Привет! Я помогу узнать курс валют и конвертировать валюту. Используйте команды:\n"
        "/available — Посмотреть доступные валюты на дату\n"
        "/rate — Узнать курс валюты на дату\n"
        "/convert — Конвертировать валюту"
    )
#--------------------------------------------------------------------------------------------------------------------------------------------------
# async def available_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
#     """
#     Обрабатывает команду /available, выводит список доступных валют на указанную дату.
#     """
#     if context.args:
#         date = context.args[0]  # Извлекаем дату из аргументов команды
#         result = available(date)  # Получаем список валют
#         await update.message.reply_text(result)
#     else:
#         await update.message.reply_text("Пожалуйста, укажите дату в формате ДД.ММ.ГГГГ. Пример: /available 01.02.2024")
from datetime import datetime

MIN_DATE = datetime(1992, 7, 1)  # Примерная дата начала публикации данных Банком России

async def available_command(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """
    Запрашивает у пользователя дату для команды /available.
    """
    await update.message.reply_text("Введите дату в формате ДД.ММ.ГГГГ:")
    return WAITING_FOR_RATE_DATE

async def get_available_date(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    date = update.message.text.strip()

    # Проверяем формат даты
    try:
        user_date = datetime.strptime(date, "%d.%m.%Y")
    except ValueError:
        await update.message.reply_text("Неверный формат даты. Убедитесь, что дата указана в формате ДД.ММ.ГГГГ:")
        return WAITING_FOR_RATE_DATE

    # Проверяем, что дата не раньше минимально доступной
    if user_date < MIN_DATE:
        await update.message.reply_text(f"Таких старых данных нет. Введите дату начиная с {MIN_DATE.strftime('%d.%m.%Y')}:")
        return WAITING_FOR_RATE_DATE

    # Проверяем, что дата не из будущего
    if user_date > datetime.now():
        await update.message.reply_text("Дата ещё не наступила. Введите корректную дату:")
        return WAITING_FOR_RATE_DATE

    # Если дата корректна, получаем список валют
    result = available(date)
    await update.message.reply_text(result)
    return ConversationHandler.END
#-------------------------------------------------------------------------------------------------------------------------------------------------------
# Минимальная доступная дата (время начала публикации данных)
MIN_DATE = datetime(1992, 7, 1)  # Примерная дата начала публикации данных Банком России

# Функция для проверки, не является ли дата раньше доступной
def is_date_valid(date_str: str) -> bool:
    try:
        date = datetime.strptime(date_str, "%d.%m.%Y")
        # Проверяем, не раньше ли введенная дата минимально доступной
        if date < MIN_DATE:
            return False
        return True
    except ValueError:
        return False

# Получение даты для курса валют
async def get_rate_date(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    date = update.message.text.strip()

    # Проверяем формат даты
    try:
        user_date = datetime.strptime(date, "%d.%m.%Y")
    except ValueError:
        await update.message.reply_text("Неверный формат даты. Убедитесь, что дата указана в формате ДД.ММ.ГГГГ:")
        return WAITING_FOR_RATE_DATE

    # Проверяем, что дата не раньше минимально доступной
    if user_date < MIN_DATE:
        await update.message.reply_text(f"Таких старых данных нет. Введите дату начиная с {MIN_DATE.strftime('%d.%m.%Y')}:")
        return WAITING_FOR_RATE_DATE

    # Проверяем, что дата не из будущего
    if user_date > datetime.now():
        await update.message.reply_text("Дата ещё не наступила. Введите корректную дату:")
        return WAITING_FOR_RATE_DATE

    # Если дата корректна, сохраняем её и запрашиваем валюту
    context.user_data["date"] = date
    await update.message.reply_text("Введите код валюты (например, USD):")
    return WAITING_FOR_RATE_CURRENCY

# Получение валюты и вывод курса
async def get_rate_currency(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    currency = update.message.text.strip().upper()  # Преобразуем ввод в верхний регистр
    context.user_data["currency_code"] = currency  # Сохраняем валюту в user_data

    date = context.user_data.get("date")  # Получаем дату из user_data
    result = exchanger(date, currency)  # Вызываем функцию для получения курса валюты

    if "не найдена" in result:  # Если валюта не найдена
        await update.message.reply_text(f"Валюта {currency} не найдена на дату {date}, воспользуйтесь функцией /available.")
        return WAITING_FOR_RATE_CURRENCY  # Снова ждем правильный ввод валюты

    await update.message.reply_text(result)
    return ConversationHandler.END  # Завершаем разговор

# Обработчик команды /convert
async def convert(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    await update.message.reply_text("Введите дату в формате ДД.ММ.ГГГГ:")
    return WAITING_FOR_CONVERT_DATE

async def get_convert_date(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    date = update.message.text.strip()

    if not is_valid_date(date):  # Проверяем формат даты
        await update.message.reply_text("Некорректная дата. Пожалуйста, введите дату в формате ДД.ММ.ГГГГ:")
        return WAITING_FOR_CONVERT_DATE  # Снова ждем правильный ввод даты

    context.user_data["date"] = date
    await update.message.reply_text("Введите количество валюты для конвертации:")
    return WAITING_FOR_CONVERT_AMOUNT

# Получение суммы для конвертации
async def get_convert_amount(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    try:
        context.user_data["amount"] = float(update.message.text)
        await update.message.reply_text("Введите код валюты, из которой будем конвертировать (например, USD):")
        return WAITING_FOR_CONVERT_FROM
    except ValueError:
        await update.message.reply_text("Некорректный ввод. Пожалуйста, введите число:")
        return WAITING_FOR_CONVERT_AMOUNT  # Снова ждем правильный ввод суммы

# Получение исходной валюты для конвертации
async def get_convert_from(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    context.user_data["from_currency"] = update.message.text.upper()
    await update.message.reply_text("Введите код валюты, в которую будем конвертировать (например, EUR):")
    return WAITING_FOR_CONVERT_TO

# Получение целевой валюты и расчет конверсии
async def get_convert_to(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    date = context.user_data["date"]
    amount = context.user_data["amount"]
    from_currency = context.user_data["from_currency"]
    to_currency = update.message.text.upper()

    # Подключаем вашу функцию для конвертации валюты (convert_currency)
    result = convert_currency(amount, from_currency, to_currency, date)
    await update.message.reply_text(result)
    return ConversationHandler.END  # Завершаем разговор

# Основная функция для запуска бота
async def main() -> None:
    TOKEN = "7833948236:AAHt_uU8ndq5Rm1tlRiIZ0b4r0UoPzdStTk"  # Замените на ваш токен
    application = Application.builder().token(TOKEN).build()

    # Добавляем ConversationHandler для команды /available
    #application.add_handler(CommandHandler("available", available_command))
    available_conversation = ConversationHandler(
    entry_points=[CommandHandler("available", available_command)],
    states={
        WAITING_FOR_RATE_DATE: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_available_date)],
    },
    fallbacks=[],
)
    application.add_handler(available_conversation)
    # Обработчик команды /start
    application.add_handler(CommandHandler("start", start))

    # Создание ConversationHandler для команды /rate
    rate_conversation = ConversationHandler(
        entry_points=[CommandHandler("rate", rate)],
        states={
            WAITING_FOR_RATE_DATE: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_rate_date)],
            WAITING_FOR_RATE_CURRENCY: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_rate_currency)],
        },
        fallbacks=[],
    )
    # Добавляем ConversationHandler для команды /rate
    application.add_handler(rate_conversation)

    # Создание ConversationHandler для команды /convert
    convert_conversation = ConversationHandler(
    entry_points=[CommandHandler("convert", convert)],
    states={
        WAITING_FOR_CONVERT_DATE: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_convert_date)],
        WAITING_FOR_CONVERT_AMOUNT: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_convert_amount)],
        WAITING_FOR_CONVERT_FROM: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_convert_from)],
        WAITING_FOR_CONVERT_TO: [MessageHandler(filters.TEXT & ~filters.COMMAND, get_convert_to)],
    },
    fallbacks=[],
)
    # Добавляем ConversationHandler для команды /convert
    application.add_handler(convert_conversation)

    # Запуск бота
    await application.run_polling()

# Запуск бота
if __name__ == "__main__":
    import nest_asyncio
    import asyncio
    nest_asyncio.apply()
    await main()
    #asyncio.run(main())

RuntimeError: Cannot close a running event loop